<a href="https://colab.research.google.com/github/Kmatsu-tokudai/TextEmotionEstimation/blob/main/TextEmotionEstimationModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E5を事前学習済みモデルとした感情推定

In [1]:
!pip install keras-core "jax[cuda]" jaxlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 61.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-cuda-nvcc-cu12
    Found existing installation: nvidia-cuda-nvcc-cu12 12.5.82
    Uninstalling nvidia-cuda-nvcc-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-nvcc-cu12-12.5.82


In [2]:
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
e5_tokenizer = AutoTokenizer.from_pretrained("Numind/e5-multilingual-sentiment_analysis")
e5_model = AutoModel.from_pretrained("Numind/e5-multilingual-sentiment_analysis")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [3]:
# 平均プーリング
def average_pool(last_hidden_states: Tensor,
                attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def extE5( input_texts):
    # Tokenize the input texts
    batch_dict = e5_tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = e5_model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)
    scores = (embeddings[:2] @ embeddings[2:].T) * 100
    emb = embeddings.to('cpu').detach().numpy().copy()
    return emb

In [8]:
# モデルのロード
import gdown
gdown.download('https://drive.google.com/uc?id=1pfiC4-waJPUBjdcoF4hDLCHNTQB_OR_m', "model.keras.weights.h5")
gdown.download('https://drive.google.com/uc?id=1SR76rllIEMNDUZi6ro2HkNXon1FdRH96', "model.keras")

!mkdir model_E5senti_em8
!mv model.keras.weights.h5 model_E5senti_em8/model.keras.weights.h5
!mv model.keras model_E5senti_em8/model.keras


Downloading...
From: https://drive.google.com/uc?id=1pfiC4-waJPUBjdcoF4hDLCHNTQB_OR_m
To: /content/model.keras.weights.h5
100%|██████████| 9.55M/9.55M [00:00<00:00, 41.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SR76rllIEMNDUZi6ro2HkNXon1FdRH96
To: /content/model.keras
100%|██████████| 9.55M/9.55M [00:00<00:00, 39.7MB/s]


In [10]:
def toStr(vecs, emos):
    strings, labelList = [], []
    for vec in vecs:
        ss = ''
        labels = []
        for i, v in enumerate(vec):
            ss += f'{v:.5f}\t'
            if v >= 0.2:
                labels.append([emos[i], v])
        ss = ss.rstrip('\t')
        strings.append(ss)
        sxs = ''
        for l in labels:
            sxs += f'{l[0]}:{l[1]} '
        sxs = sxs.rstrip(' ')
        labelList.append(sxs)

In [37]:
import os, sys, re, glob
os.environ['KERAS_BACKEND'] = 'jax'
# JAXバックエンドでのメモリ断片化を回避
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
import numpy as np
from keras_core import datasets, layers, models
import keras_core as keras
from keras_core.callbacks import EarlyStopping
import pandas as pd

mdir = './model_E5senti_em8'
loaded_model = keras.saving.load_model(f"{mdir}/model.keras")
loaded_model.load_weights(f"{mdir}/model.keras.weights.h5")
loaded_model.summary()

emos = ['Joy', 'Sadness', 'Anticipation', 'Surprise', 'Anger', 'Fear', 'Disgust', 'Trust']


def toStr(vecs, emos):
    strings, labelList = [], []
    for vec in vecs:
        ss = ''
        labels = []
        for i, v in enumerate(vec):
            ss += f'{v:.5f}\t'
            if v >= 0.2:
                labels.append([emos[i], v])
        ss = ss.rstrip('\t')
        strings.append(ss)
        sxs = ''
        for l in labels:
            sxs += f'{l[0]}:{l[1]} '
        sxs = sxs.rstrip(' ')
        labelList.append(sxs)

    return strings, labelList


sentences = ['今日から長期休暇だぁーーー！！！', 'めっちゃ楽しみ♡']
vL = extE5(sentences)
vL = np.array(vL)

vL = np.reshape(vL, (len(vL), 768))
r = loaded_model.predict(vL)
rs, ls = toStr(r, emos)

for i, (l,r) in enumerate(zip(ls, rs)):
    print(l)
    print(f"[{sentences[i]}]\tResult: : {emos[np.argmax(r)]}")




Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer1 (Dense)            │ (256, 512)                │    393,728 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ middle_layer1 (Dense)           │ (256, 768)                │    393,984 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout (Dropout)               │ (256, 768)                │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ output_layer1 (Dense)           │ (256, 8)                  │      6,152 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 2,381,594 (9.09 MB)

 Trainable params: 793,864 (3.03 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,587,730 (6.06 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
Joy:0.603644609451294 Sadness:0.39135220646858215 Anticipation:0.38978999853134155 Surprise:0.38142767548561096 Anger:0.2886408269405365 Fear:0.29910480976104736 Disgust:0.30761730670928955 Trust:0.2677551507949829
[今日から長期休暇だぁーーー！！！]	Result: : Joy
Joy:0.6509655714035034 Sadness:0.25046539306640625 Anticipation:0.4483218789100647 Surprise:0.3964574635028839 Anger:0.29757410287857056 Fear:0.27986904978752136 Disgust:0.2425859570503235 Trust:0.4983934164047241
[めっちゃ楽しみ♡]	Result: : Joy


# RosettaJA を用いた感情推定

In [18]:
# モデルのダウンロード
gdown.download("https://drive.google.com/uc?id=1cl5lOLOmmKSLGeMBlg-Xro2C5nyuKyK3", 'model.keras')
gdown.download("https://drive.google.com/uc?id=1YkKGaEbZUAAK4oG-x_adFHqzQ9szAPXW", 'model.keras.weights.h5')

!mkdir model_RosettaJA_em8
!mv model.keras.weights.h5 model_RosettaJA_em8/model.keras.weights.h5
!mv model.keras model_RosettaJA_em8/model.keras

Downloading...
From: https://drive.google.com/uc?id=1cl5lOLOmmKSLGeMBlg-Xro2C5nyuKyK3
To: /content/model.keras
100%|██████████| 5.55M/5.55M [00:00<00:00, 193MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YkKGaEbZUAAK4oG-x_adFHqzQ9szAPXW
To: /content/model.keras.weights.h5
100%|██████████| 5.55M/5.55M [00:00<00:00, 32.0MB/s]


In [19]:
# pkshatech/RoSEtta-base-ja(テキスト特徴量)用
from sentence_transformers import SentenceTransformer

rosetta_model = SentenceTransformer("pkshatech/RoSEtta-base-ja",trust_remote_code=True)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.63k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/975 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [30]:
# Rosetta（テキストベクトル）の抽出
def extRosetta( input_texts):
    embeddings = rosetta_model.encode( input_texts, convert_to_tensor=True)
    return embeddings

In [39]:
m_rosedir = './model_RosettaJA_em8'
loaded_model = keras.saving.load_model(f"{m_rosedir}/model.keras")
loaded_model.load_weights(f"{m_rosedir}/model.keras.weights.h5")
loaded_model.summary()

emos = ['Joy', 'Sadness', 'Anticipation', 'Surprise', 'Anger', 'Fear', 'Disgust', 'Trust']

sentences = ['今日から長期休暇だぁーーー！！！', 'めっちゃ楽しみ♡']
vL = extRosetta(sentences)
vL = vL.to('cpu').detach().numpy().copy()

vL = np.reshape(vL, (len(vL), 768))
print(vL.shape)
res = loaded_model.predict(vL)
rs, ls = toStr(res, emos)

for i, (l,r) in enumerate(zip(ls, rs)):
    print(l)
    print(f"[{sentences[i]}]\tResult: : {emos[np.argmax(r)]}")


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer1 (Dense)            │ (512, 512)                │    393,728 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout (Dropout)               │ (512, 512)                │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ middle_layer3 (Dense)           │ (512, 128)                │     65,664 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_1 (Dropout)             │ (512, 128)                │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ output_layer1 (Dense)           │ (512, 8)                  │      1,032 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 1,381,274 (5.27 MB)

 Trainable params: 460,424 (1.76 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 920,850 (3.51 MB)

(2, 768)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Sadness:0.9575691223144531
[今日から長期休暇だぁーーー！！！]	Result: : Joy
Joy:1.0 Trust:0.9647204279899597
[めっちゃ楽しみ♡]	Result: : Joy
